# Exemplary Usage of Scoring Rules
This notebook shows the usage of different scoring functions with exemplary data.

In [1]:
import timeit # timeit for measuring runtime of approaches

In [2]:
import scoring as scoring # the scoring module
import numpy as np

### Test data

In [3]:
# Exemplary ground truth observations as array
observations_test = np.array([4,7,4,6,2,1,3,8])

# Exemplary point forecasts
point_dict_test = {
    0.1: np.array([2, 4.7, 5.2 , 9.6, 1.8, -2  , 0.4, 8.8]),
    0.2: np.array([2, 4.7, 5.2 , 9.6, 1.8, -2  , 0.4, 8.8]),
    0.5: np.array([2, 4.7, 5.2 , 9.6, 1.8, -2  , 0.4, 8.8]),
    0.8: np.array([2, 4.7, 5.2 , 9.6, 1.8, -2  , 0.4, 8.8]),
    0.9: np.array([2, 4.7, 5.2 , 9.6, 1.8, -2  , 0.4, 8.8]),
}

# Exemplary quantile forecasts
quantile_dict_test = {
    0.1: np.array([2, 3  , 5   , 9  , 1  , -3  , 0.2, 8.7]),
    0.2: np.array([2, 4.6, 5   , 9.4, 1.4, -2  , 0.4, 8.8]),
    0.5: np.array([2, 4.7, 5.2 , 9.6, 1.8, -2  , 0.4, 8.8]),
    0.8: np.array([4, 4.8, 5.7 , 12 , 4.3, -1.5, 2  , 8.9]),
    0.9: np.array([5, 5  , 7   , 13 , 5  , -1  , 3  , 9])
}

# Exemplary alpha level
alpha_test=0.2

### Interval Score

Compute interval scores by providing a left and right quantile.

In [4]:
scoring.interval_score(observations_test,alpha_test,q_left=quantile_dict_test[0.1],q_right=quantile_dict_test[0.9])

(array([ 3. , 22. , 12. , 34. ,  4. , 22. ,  2.8,  7.3]),
 array([3. , 2. , 2. , 4. , 4. , 2. , 2.8, 0.3]),
 array([ 0., 20., 10., 30.,  0., 20.,  0.,  7.]))

Compute interval scores by providing a dictionary of quantiles. The quantiles needed for the alpha level specified are automatically selected from the dictionary.

In [5]:
scoring.interval_score(observations_test,alpha_test,q_dict=quantile_dict_test)

(array([ 3. , 22. , 12. , 34. ,  4. , 22. ,  2.8,  7.3]),
 array([3. , 2. , 2. , 4. , 4. , 2. , 2.8, 0.3]),
 array([ 0., 20., 10., 30.,  0., 20.,  0.,  7.]))

Test if the percentage version of the interval score works:
We use the (1-[alpha=1])-interval, i.e. the median, for the interval score, set `percentage=True` and check whether this is two times the absolute percentage error of the median.

In [6]:
print(scoring.interval_score(observations_test,1,q_dict=quantile_dict_test,percent=True)[0])
print(2*np.abs((observations_test-quantile_dict_test[0.5])/observations_test))

[1.         0.65714286 0.6        1.2        0.2        6.
 1.73333333 0.2       ]
[1.         0.65714286 0.6        1.2        0.2        6.
 1.73333333 0.2       ]


### Weighted Interval Score

Compute weighted interval scores by providing a dictionary of quantiles and a list of alpha levels and corresponding weights.

There are two implementations, a simple one based on iterated computation of interval scores and a faster one based on joint computation of all interval scores. They yield the same results, so the faster one is preferrable and the basic one is only provided for understanding purposes.

In [7]:
print(scoring.weighted_interval_score(observations_test,alphas=[0.2,0.4],weights=[2,5],q_dict=quantile_dict_test))
print(scoring.weighted_interval_score_fast(observations_test,alphas=[0.2,0.4],weights=[2,5],q_dict=quantile_dict_test))

(array([ 2.28571429, 14.28571429,  7.5       , 23.71428571,  3.21428571,
       15.57142857,  5.51428571,  5.01428571]), array([2.28571429, 0.71428571, 1.07142857, 3.        , 3.21428571,
       0.92857143, 1.94285714, 0.15714286]), array([ 0.        , 13.57142857,  6.42857143, 20.71428571,  0.        ,
       14.64285714,  3.57142857,  4.85714286]))
(array([ 2.28571429, 14.28571429,  7.5       , 23.71428571,  3.21428571,
       15.57142857,  5.51428571,  5.01428571]), array([2.28571429, 0.71428571, 1.07142857, 3.        , 3.21428571,
       0.92857143, 1.94285714, 0.15714286]), array([ 0.        , 13.57142857,  6.42857143, 20.71428571,  0.        ,
       14.64285714,  3.57142857,  4.85714286]))


Compute weighted interval scores by providing a dictionary of quantiles and a list of alpha levels, but no corresponding weights. This means setting `weights=alphas/2`, yielding an approximation of the CRPS.

In [8]:
scoring.weighted_interval_score_fast(observations_test,alphas=[0.2,0.4],weights=None,q_dict=quantile_dict_test)

(array([ 2.33333333, 14.8       ,  7.8       , 24.4       ,  3.26666667,
        16.        ,  5.33333333,  5.16666667]),
 array([2.33333333, 0.8       , 1.13333333, 3.06666667, 3.26666667,
        1.        , 2.        , 0.16666667]),
 array([ 0.        , 14.        ,  6.66666667, 21.33333333,  0.        ,
        15.        ,  3.33333333,  5.        ]))

Compute percentage version of the weighted interval scores (once with just the median, equal to the simple interval scorce, and once with several intervals).

In [9]:
print(scoring.weighted_interval_score_fast(observations_test,alphas=[1],weights=None,q_dict=point_dict_test,percent=True)[0])
print(scoring.weighted_interval_score_fast(observations_test,alphas=[0.2,0.4,1],weights=None,q_dict=point_dict_test,percent=True)[0])

[1.         0.65714286 0.6        1.2        0.2        6.
 1.73333333 0.2       ]
[ 1.875       1.23214286  1.125       2.25        0.375      11.25
  3.25        0.375     ]


#### Compare runtimes of weighted interval score methods

Runtimes of simple implementation for different sizes

In [10]:
print({n: min(timeit.repeat(lambda: scoring.weighted_interval_score(observations_test,alphas=[0.2]*n,weights=None,q_dict=quantile_dict_test),repeat=100,number=100)) for n in [2,5,10,20]})

{2: 0.007953100000000823, 5: 0.014574499999999269, 10: 0.02527900000000116, 20: 0.04693550000000002}


Runtimes of fast implementation for different sizes

In [11]:
print({n: min(timeit.repeat(lambda: scoring.weighted_interval_score_fast(observations_test,alphas=[0.2]*2,weights=None,q_dict=quantile_dict_test),repeat=100,number=100)) for n in [2,5,10,20,40,80]})

{2: 0.008790000000001186, 5: 0.00875909999999891, 10: 0.008776400000002127, 20: 0.008793000000000717, 40: 0.008763800000000543, 80: 0.008803100000001507}


As can be seen, the fast implementation is in fact quicker for more than 2 or 3 intervals. Its runtime stays almost constant.

### Outside Interval Count

Count number of times the true observation was outside an interval.

In [12]:
print(scoring.outside_interval(observations_test,lower=quantile_dict_test[0.1],upper=quantile_dict_test[0.9]))

[0 1 1 1 0 1 0 1]


### Interval Consistency Score

Compute adapted variant of the interval score which measures the consistency of updated intervals over time.

The interval consistency score does not evaluate the sharpness of the intervals. It only penalizes the difference between old and new forecasted intervals when the new interval is (partially) outside the old interval.
Ideally, updated predicted intervals would always be within the previous estimates of the interval, yielding
a score of zero (best). The underlying logic is that the old forecasts have been made with less perfect knowledge than the new ones and should reflect this epistemic uncertainty in wider prediction intervals.

In [13]:
print(scoring.interval_consistency_score(lower_old=quantile_dict_test[0.1],upper_old=quantile_dict_test[0.9],lower_new=quantile_dict_test[0.1],upper_new=quantile_dict_test[0.8]))

[0. 0. 0. 0. 0. 0. 0. 0.]


In [14]:
print(scoring.interval_consistency_score(lower_old=quantile_dict_test[0.1],upper_old=quantile_dict_test[0.8],lower_new=quantile_dict_test[0.1],upper_new=quantile_dict_test[0.9]))

[1.  0.2 1.3 1.  0.7 0.5 1.  0.1]
